In [3]:
import pandas as pd
import warnings
from pandas import Series, DataFrame
warnings.filterwarnings("ignore")

In [2]:
#data_offline = pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')

In [5]:
tr2=cutoff_time(20160200,20160500,data_online).drop_duplicates()


In [11]:
tr1=cutoff_time(20160100,20160400,data_online).drop_duplicates()
te1=cutoff_time(20160400,20160500,data_online).drop_duplicates()
tr2=cutoff_time(20160200,20160500,data_online).drop_duplicates()
te2=cutoff_time(20160500,20160600,data_online).drop_duplicates()
train=cutoff_time(20160100,20160600,data_online).drop_duplicates()
test=cutoff_time(20160600,20160700,data_online).drop_duplicates()

In [4]:
def get_date_number(s):
    if s=='null':
        return 0
    else:
        return int(s)
def judge_cut_off(start_time,end_time,s):
    return int(s>start_time and s<end_time)
 
    
def cutoff_time(start_time,end_time,input_data):
    input_data['date']=input_data['Date'].map(get_date_number)
    input_data['dr']=input_data['Date_received'].map(get_date_number)
    input_data['jd']=input_data['date']+input_data['dr']
    input_data['judge']=input_data['jd'].map(lambda x:judge_cut_off(start_time,end_time,x))
    input_data['judge1']=input_data['date'].map(lambda x:int(x>0 and x<end_time))                             
    input_data['finaljd']=input_data['judge']+input_data['judge1']
    output=input_data[input_data['finaljd']>0].drop(['date','dr','jd','judge','judge1','finaljd'], axis=1)
    return output
    
    
    
    

In [6]:
import time, datetime
def sent_coupon(value):
    if value != 'null':
        return 1
    else: 
        return 0
    
def used_coupon(a,b):
    if a == 1 and b != 'null':
        return 1
    else:
        return 0
    
def get_discount_rate(s):
    if s != 'null':
        if len(s.split(':')) > 1:
            return (float(s.split(':')[0]) - float(s.split(':')[1])) / float(
                s.split(':')[0])
        elif s == 'fixed':
            return s
        else:
            return float(s)
    else:
        return 'null'


def get_coupon_requirement(s):
    if s != 'null':
        if len(s.split(':')) > 1:
            return s.split(':')[0]
        else:
            return 0
    else:
        return 0


def is_direct_use(s):
    if (len(s.split(':')) == 1) & (s != 'null'):
        return 1
    else:
        return 0


def get_week_day(s):
    if s != 'null':
        date_time = datetime.datetime.strptime(s, '%Y%m%d')
        return date_time.weekday()
    else:
        return 'null'


def get_month_day(s):
    if s != 'null':
        return int(s[-2:])
    else:
        return 'null'

In [7]:
def get_coupon_feature_online(input_data):
    a = input_data[['Merchant_id', 'Coupon_id']]
    mask = input_data['Coupon_id'] != 'null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id', 'Tot_coupon_sent']
    input_data1 = pd.merge(
        input_data, num_coupon, on='Merchant_id', how='left')
    mask2 = (input_data1['Date'] != 'null') & (input_data1['Coupon_id'] !=
                                               'null')
    b = input_data1[mask2].groupby(
        ['Merchant_id'])['Action'].count().reset_index()
    b.columns = ['Merchant_id', 'Num_coupon_used']
    input_data2 = pd.merge(input_data1, b, on='Merchant_id', how='left')
    l = input_data2[['Merchant_id', 'Coupon_id', 'Action']].groupby(
        ['Merchant_id', 'Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id'] != 'null'
    l2 = l[mask3]
    l2['num_type'] = l2['Coupon_id'].apply(lambda x: int(x > 0))
    l3 = l2.groupby('Merchant_id')['num_type'].count()
    l4 = pd.DataFrame(l3).reset_index()
    input_data3 = pd.merge(input_data2, l4, on='Merchant_id', how='left')
    couponmask = (input_data3['Date'] != 'null') & (input_data3['Coupon_id'] !=
                                                    'null')
    coupon_used = input_data3[couponmask]
    input_data3['coupon_requirement'] = input_data3['Discount_rate'].map(
        get_coupon_requirement)
    input_data3['equivalent_discount'] = input_data3['Discount_rate'].map(
        get_discount_rate)
    input_data3['direct_use_coupon'] = input_data3['Discount_rate'].map(
        is_direct_use)
    input_data3['Date_received_week'] = input_data3['Date_received'].map(
        get_week_day)
    input_data3['Date_received_month'] = input_data3['Date_received'].map(
        get_month_day)
    output = input_data3[input_data3['Coupon_id'] != 'null']
    output.drop(['Discount_rate', 'Date_received', 'Date'], axis=1, inplace=True)
    print 'Coupon Fearures completed'
    return output

In [8]:
def get_custMerch(data):
    data['Coupon Received'] = data['Coupon_id'].map(sent_coupon)
    data['Coupon Used'] = data.apply(lambda x: used_coupon(x['Coupon Received'], x['Date']), axis = 1)
    grouped = data.groupby(['User_id', 'Merchant_id'], as_index = False)[['Coupon Received','Coupon Used']].agg(sum)
    grouped["Customer's Total Usage Fraction for Merchant"] = grouped['Coupon Used'] / grouped['Coupon Received'].fillna(0)
    grouped["Customer's Total Usage for Merchant's Coupons"] = grouped["Coupon Used"]
    grouped["Customer's Total Received by Merchant"] = grouped["Coupon Received"]
    del grouped["Coupon Used"]
    del grouped["Coupon Received"]
    merge0 = grouped.merge(data, on = ['User_id','Merchant_id'] )
    receive_frac = data.groupby(['User_id', 'Merchant_id'])[['Coupon Received']].agg(sum)/data.groupby(['User_id'])[['Coupon Received']].agg(sum)
    receive_frac = receive_frac.reset_index()
    merged1 = receive_frac.merge(merge0, on = ['User_id','Merchant_id'])
    merged1["Customer's Inbox Share by Merchant"] = merged1['Coupon Received_x'].fillna(0)
    del merged1['Coupon Received_x']
    merged1['Coupon Received'] = merged1['Coupon Received_y']
    del merged1['Coupon Received_y']
    
    # Note: this next feature is really more merchant-specific
    merchant_group = data.groupby(['Merchant_id', 'User_id'])[['Coupon Received', 'Coupon Used']].agg(sum)
    by_merchant = merchant_group.groupby(['Merchant_id'])[['Coupon Received', 'Coupon Used']].agg(sum)
    by_merchant["Merchant's Total Usage Fraction"] = by_merchant['Coupon Used'] / by_merchant['Coupon Received']
    by_merchant["Merchant's Total Usage Fraction"] = by_merchant["Merchant's Total Usage Fraction"].fillna(0)
    del by_merchant['Coupon Received'], by_merchant['Coupon Used']
    by_merchant = by_merchant.reset_index()
    finalMerge= merged1.merge(by_merchant, on = ['Merchant_id'])
    print'Cust-Merchant features completed'
    
    return finalMerge

In [13]:
def get_coupon_requirement(s):
    if s != 'null':
        if len(s.split(':')) > 1:
            return int(s.split(':')[0])
        else:
            return 0
    else:
        return 0

In [14]:
def get_online_user_table(data_online):

    #Number of time this customer gets a coupon online
    a = data_online[['User_id', 'Coupon_id']]
    mask = data_online['Coupon_id'] != 'null'
    num_coupon = a[mask].groupby(['User_id']).count().reset_index()
    num_coupon.columns = ['User_id', 'Tot_coupon_got']
    data_online1 = pd.merge(data_online, num_coupon, on='User_id', how='left')

    #Number of time this customer uses a coupon
    mask2 = (data_online1['Date'] != 'null') & (data_online1['Coupon_id'] !=
                                                'null')
    b = data_online1[mask2].groupby(
        ['User_id'])['Action'].count().reset_index()
    b.columns = ['User_id', 'Num_coupon_used']
    data_online2 = pd.merge(data_online1, b, on='User_id', how='left')

    #The average amount of coupon the customer got a week.
    #The training set is collected from Jan. 1st 2016 to June 30th 2016. 26 weeks in total
    def to_float(s):
        return float(s)

    data_online3 = data_online2
    data_online3['Tot_coupon_got'] = data_online3['Tot_coupon_got'].map(
        to_float)
    data_online3['Avg_coupon_got'] = data_online3['Tot_coupon_got'] * 1 / 26

    #generate customer table
    customer_columns = data_online3[[
        'User_id', 'Tot_coupon_got', 'Num_coupon_used', 'Avg_coupon_got'
    ]]
    customer_online_table = customer_columns.drop_duplicates()
    data_online['money'] = data_online['Discount_rate'].map(
        get_coupon_requirement)
    out1=data_online[data_online['money']>0].groupby('User_id')['money'].mean().reset_index()
    output = pd.merge(customer_online_table,out1, on='User_id')
    print 'User features completed'
    return output

In [2]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [ ]:
QDA()

In [15]:
TR1=get_online_user_table(tr2)

User features completed


In [16]:
TR1=TR1.fillna(0)

In [17]:
TR1['Tot_coupon_got']=TR1['Tot_coupon_got'].map(lambda x:int(x))
TR1['Num_coupon_used']=TR1['Num_coupon_used'].map(lambda x:int(x))
TR1['money']=TR1['money'].map(lambda x:int(x))

In [18]:
TR1['Avg_use_rate']=TR1['Num_coupon_used']/TR1['Tot_coupon_got']

In [19]:
TR1.head()

,User_id,Tot_coupon_got,Num_coupon_used,Avg_coupon_got,money,Avg_use_rate
0,13740231,5,0,0.192308,520,0.00
1,15137031,4,0,0.153846,170,0.00
2,13332831,12,0,0.461538,150,0.00
3,14219799,1,0,0.038462,5,0.00
4,12518031,8,2,0.307692,175,0.25


In [20]:
TR1.to_csv('TR2_user.csv',index=False)

In [23]:
import webbrowser,sys
webbrowser.open('https://www.youtube.com/watch?v=NiHF-cwto_A')

True

In [8]:
def get_online_merchant_table(data_online):
    #get num_coupon for each user
    
    a=data_online[['Merchant_id','Coupon_id']]
    mask = data_online['Coupon_id']!='null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id','Tot_coupon_sent']
    data_online1 = pd.merge(data_online, num_coupon, on = 'Merchant_id', how='left')

    #get num coupon used for each user
    mask2 = (data_online1['Date']!= 'null') & (data_online1['Coupon_id']!= 'null')
    b=data_online1[mask2].groupby(['Merchant_id'])['Action'].count().reset_index()
    b.columns = ['Merchant_id','Num_coupon_used']
    data_online2 = pd.merge(data_online1, b, on='Merchant_id', how='left')

    #get total number of coupons type for each user
    l= data_online2[['Merchant_id','Coupon_id','Action']].groupby(['Merchant_id','Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id']!='null'
    l2=l[mask3]

    def num_type(s):
        if s>0:
            return 1

    l2['num_type']=l2['Coupon_id'].apply(num_type)
    l3 =l2.groupby('Merchant_id')['num_type'].count()
    l4=pd.DataFrame(l3).reset_index()
    data_online3= pd.merge(data_online2, l4, on='Merchant_id',how='left')

    #generate merchant table
    no_userid= data_online3[['Merchant_id','Tot_coupon_sent','Num_coupon_used','num_type']]
    merchant_online_table = no_userid.drop_duplicates()
    print 'Merchant features completed'
    return merchant_online_table

In [9]:
def pipeline(input_data):
    M=get_online_merchant_table(input_data)
    CU=get_online_user_table(input_data)
    CO=get_coupon_feature_online(input_data)
    CM=get_custMerch(input_data)
    MG1=CO.merge(M,left_on='Merchant_id',right_on='Merchant_id',how='left')
    print 'Merge Completed1/3'
    MG2=MG1.merge(CU,left_on='User_id',right_on='User_id',how='left')
    print 'Merge Completed2/3'
    MG1=MG2.merge(CM,left_on=['User_id','Merchant_id'],right_on=['User_id','Merchant_id'],how='left')
    print 'Merge Completed3/3'
    return MG1
    

# Code to run:

In [ ]:
tr1=cutoff_time(20160100,20160400,data_online)
te1=cutoff_time(20160400,20160500,data_online)
tr2=cutoff_time(20160200,20160500,data_online)
te2=cutoff_time(20160500,20160600,data_online)
train=cutoff_time(20160100,20160600,data_online)
test=cutoff_time(20160600,20160700,data_online)
TR1=pipeline(tr1)
TE1=pipeline(te1)
TR2=pipeline(tr2)
TE2=pipeline(te2)
TRAIN=pipeline(train)
TEST=pipeline(test)



## For offline data

In [82]:
def get_coupon_feature_offline(input_data):
    a = input_data[['Merchant_id', 'Coupon_id']]
    mask = input_data['Coupon_id'] != 'null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id', 'Tot_coupon_sent']
    input_data1 = pd.merge(input_data, num_coupon, on='Merchant_id')
    mask2 = (input_data1['Date'] != 'null') & (input_data1['Coupon_id'] !=
                                                 'null')
    b = input_data1[mask2].groupby(
        ['Merchant_id'])['Coupon_id'].count().reset_index()
    b.columns = ['Merchant_id', 'Num_coupon_used']
    input_data2 = pd.merge(input_data1, b, on='Merchant_id')
    l = input_data2[['Merchant_id', 'Coupon_id']].groupby(
        ['Merchant_id', 'Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id'] != 'null'
    l2 = l[mask3]
    l2['num_type'] = l2['Coupon_id'].apply(lambda x:int(x>0))
    l3 = l2.groupby('Merchant_id')['num_type'].count()
    l4 = pd.DataFrame(l3).reset_index()
    input_data3 = pd.merge(input_data2, l4, on='Merchant_id')
    null_mask = input_data3['Distance'] != 'null'
    input_data4 = input_data3[null_mask]
    input_data4['Distance'] = input_data4['Distance'].map(lambda x: float(x))
    input_data5 = pd.DataFrame(
        input_data4.groupby('Merchant_id')['Distance'].mean()).reset_index()
    input_data6 = pd.merge(input_data3, input_data5, on='Merchant_id')
    input_data6 = input_data6.rename(columns={'Distance_y': 'Avg_Distance'})
    input_data6['equivalent_discount'] = input_data6['Discount_rate'].map(
        get_discount_rate)
    input_data6['direct_use_coupon'] = input_data6['Discount_rate'].map(
        is_direct_use)
    input_data6['coupon_requirement'] = input_data6['Discount_rate'].map(
        get_coupon_requirement)
    input_data6['Date_received_week'] = input_data6['Date_received'].map(
        get_week_day)
    input_data6['Date_received_month'] = input_data6['Date_received'].map(
        get_month_day)
    output=input_data6[input_data6['Coupon_id']!='null'] 
    output.drop(['Discount_rate', 'Date_received', 'Date'], axis=1, inplace=True)
    return output

In [83]:
b=get_coupon_feature_offline(data_offline)

In [85]:
b.head()

,User_id,Merchant_id,Coupon_id,Distance_x,Tot_coupon_sent,Num_coupon_used,num_type,Avg_Distance,equivalent_discount,direct_use_coupon,coupon_requirement,Date_received_week,Date_received_month
1,1439408,2632,8591,0,43,4,2,1.54,0.95,0,20,2,17
2,1439408,2632,1078,0,43,4,2,1.54,0.95,0,20,5,19
3,1439408,2632,8591,0,43,4,2,1.54,0.95,0,20,0,13
5,1439408,2632,8591,0,43,4,2,1.54,0.95,0,20,0,16
6,2082569,2632,1078,3,43,4,2,1.54,0.95,0,20,4,25


In [4]:
x=pd.read_csv('full_train.csv')

In [14]:
x_money=x[x['money'].notnull()]

In [15]:
x_money['money_quantile']= pd.qcut(x_money['money'],10,labels=[1,2,3,4,5,6,7,8,9,10])

In [23]:
x_money.groupby('money_quantile')['Coupon Used_x'].mean()

money_quantile
1     0.239523
2     0.169279
3     0.222319
4     0.245433
5     0.222139
6     0.169694
7     0.226525
8     0.216782
9     0.223134
10    0.243012
Name: Coupon Used_x, dtype: float64

In [20]:
%matplotlib inline

In [24]:
x_money.head()

,coupon_requirement,equivalent_discount,Date_received_week,Date_received_month,Tot_coupon_sent,Num_coupon_used,num_type,Customer's Total Usage Fraction for Merchant,Customer's Total Usage for Merchant's Coupons,Customer's Total Received by Merchant,Merchant's Total Usage Fraction,Customer's Inbox Share by Merchant,Difference,money,Coupon Used_x,money_quantile
0,500,0.9,4,13,35140,0.0,123,0.0,0,6,0.210273,1.00,3.0,520.0,0,10
1,50,0.9,3,17,1607,0.0,11,0.0,0,1,0.168015,0.25,18.0,170.0,0,8
2,30,0.966666667,1,5,248,0.0,4,0.0,0,1,0.088710,0.25,18.0,170.0,0,8
3,300,0.833333333,2,6,35140,0.0,123,0.0,0,2,0.210273,0.50,18.0,170.0,0,8
4,300,0.9,3,17,35140,0.0,123,0.0,0,2,0.210273,0.50,18.0,170.0,0,8


In [26]:
x_RFM=x_money[x_money['Difference'].notnull()]

In [27]:
x_RFM['Recency_quantile']= pd.qcut(x_RFM['Difference'],10,labels=[1,2,3,4,5,6,7,8,9,10])

In [31]:
DataFrame(x_RFM.groupby(['money_quantile','Recency_quantile'])['Coupon Used_x'].mean()).to_csv('RFM.csv')

In [32]:
x_RFM.head()

,coupon_requirement,equivalent_discount,Date_received_week,Date_received_month,Tot_coupon_sent,Num_coupon_used,num_type,Customer's Total Usage Fraction for Merchant,Customer's Total Usage for Merchant's Coupons,Customer's Total Received by Merchant,Merchant's Total Usage Fraction,Customer's Inbox Share by Merchant,Difference,money,Coupon Used_x,money_quantile,Recency_quantile
0,500,0.9,4,13,35140,0.0,123,0.0,0,6,0.210273,1.00,3.0,520.0,0,10,1
1,50,0.9,3,17,1607,0.0,11,0.0,0,1,0.168015,0.25,18.0,170.0,0,8,5
2,30,0.966666667,1,5,248,0.0,4,0.0,0,1,0.088710,0.25,18.0,170.0,0,8,5
3,300,0.833333333,2,6,35140,0.0,123,0.0,0,2,0.210273,0.50,18.0,170.0,0,8,5
4,300,0.9,3,17,35140,0.0,123,0.0,0,2,0.210273,0.50,18.0,170.0,0,8,5


In [33]:
from sklearn.svm import SVC
SVC()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RandomForestClassifier()